In [1]:
!git clone https://github.com/danilkos00/sentiment-analysis.git
%cd sentiment-analysis
!pip install -r requirements.txt -qq

Cloning into 'sentiment-analysis'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 40 (delta 14), reused 36 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 4.86 MiB | 8.83 MiB/s, done.
Resolving deltas: 100% (14/14), done.
/content/sentiment-analysis
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s e

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
df = pd.read_csv('./data/Womens_Clothing_E-Commerce_Reviews.csv')
df = df[['Title', 'Review Text', 'Recommended IND']]
df = df.fillna('')
df['Review Text'] = df['Title'] + ' ' + df['Review Text']
df.drop('Title', axis=1, inplace=True)
df.columns = ['text', 'label']
df.head()

,text,label
0,Absolutely wonderful - silky and sexy and com...,1
1,Love this dress! it's sooo pretty. i happen...,1
2,Some major design flaws I had such high hopes ...,0
3,"My favorite buy! I love, love, love this jumps...",1
4,Flattering shirt This shirt is very flattering...,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    23486 non-null  object
 1   label   23486 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 367.1+ KB


In [5]:
df['label'].value_counts()

,count
label,
1,19314
0,4172


In [11]:
from src.preprocessing import text_preprocessing

for i in np.random.randint(0, df.shape[0], 2):
    print('Original: ' + df.text[i])
    print('Processed: ' + text_preprocessing(df.text[i]) + '\n')

Original: Love style, need different color I love this tshirt, but the speckled grey looks more like roof shingles in person. i'm going to try to exchange it for a different color.
Processed: love style need different color I love this tshirt but the speckle grey look more like roof shingle in person I m go to try to exchange it for a different color

Original:  I own quite a few deletta tops, and this one nails it. a great staple in a pretty color (i purchased the green). peplum tops can be hit or miss for me and often come up short or balloon out too much. this one is so flattering. i initially ordered a medium (5'8", 145) but exchanged for a small when i found the medium to be a little roomy on me. the small is perfect. the top is made of a medium weight material and is slightly fitted, nipping in at the waist. the "skirt" piece falls nicely and is l
Processed: I own quite a few deletta top and this one nail it a great staple in a pretty color I purchase the green peplum top can be 

In [7]:
import os

if not os.path.exists('./data/df_cleared.csv'):
    tqdm.pandas()
    df_cleared = df.copy()
    df_cleared['text'] = df_cleared['text'].progress_apply(text_preprocessing)
    df_cleared.to_csv("./data/df_cleared.csv", index=False)

In [8]:
from datasets import Dataset
import torch
from torch.utils.data import DataLoader
from transformers import get_scheduler
from sklearn.model_selection import train_test_split

In [9]:
full_df = pd.read_csv("./data/df_cleared.csv")
full_df.dropna(inplace=True)
full_df.head()

,text,label
0,absolutely wonderful silky and sexy and comfor...,1
1,love this dress it s sooo pretty I happen to f...,1
2,some major design flaw I have such high hope f...,0
3,my favorite buy I love love love this jumpsuit...,1
4,flatter shirt this shirt be very flatter to al...,1


In [10]:
from src.model import load_tokenizer, load_model, train_model

train, test = train_test_split(full_df, test_size=0.25, )
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

tokenizer = load_tokenizer('distilbert-base-uncased-finetuned-sst-2-english')
model = load_model('distilbert-base-uncased-finetuned-sst-2-english', num_labels=2)
model

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [27]:
from src.preprocessing import df_proc

tokenized_train = df_proc(train, tokenizer)
tokenized_test = df_proc(test, tokenizer)

Map:   0%|          | 0/16981 [00:00<?, ? examples/s]

Map:   0%|          | 0/5661 [00:00<?, ? examples/s]

In [29]:
train_dataloader = DataLoader(tokenized_train, batch_size=16, shuffle=True)
test_dataloader = DataLoader(tokenized_test, batch_size=8)
train_length = len(train)
test_length = len(test)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6)

num_epochs = 5
num_train_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(name='linear',
                             optimizer=optimizer,
                             num_warmup_steps=0,
                             num_training_steps=num_train_steps)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device);

In [13]:
train_model(model, train_dataloader, test_dataloader, optimizer, lr_scheduler, train_length, test_length, device, num_epochs)

Epoch 1/2, Training:


Train Loss: 0.3299


Validation:


Val Loss: 0.2574, f1 score: {'f1': 0.9331635900700191}
Epoch 2/2, Training:


Train Loss: 0.2567
Validation:


Val Loss: 0.2435, f1 score: {'f1': 0.9362786745964317}


In [12]:
import gdown


url = 'https://drive.google.com/uc?id=1snKee0oLYAKJ-F5sTFZmh7qpEZrNU-Xg'
output_path = './model.pt'

gdown.download(url, output_path, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1snKee0oLYAKJ-F5sTFZmh7qpEZrNU-Xg
From (redirected): https://drive.google.com/uc?id=1snKee0oLYAKJ-F5sTFZmh7qpEZrNU-Xg&confirm=t&uuid=444e5f81-b56a-40b5-a4f9-3a2970915139
To: /content/sentiment-analysis/model.pt
100%|██████████| 268M/268M [00:03<00:00, 76.1MB/s]


'./model.pt'

In [15]:
from src.model import load_tokenizer, load_model, infer_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = load_tokenizer("distilbert-base-uncased-finetuned-sst-2-english")
model = load_model('distilbert-base-uncased-finetuned-sst-2-english', num_labels=2)
model.load_state_dict(torch.load('./model.pt', map_location=device))
model.to(device)

infer_model(model, tokenizer, df, device, num_samples=10)

,text,predict,label
0,darling top run large and long I be a solid si...,1,1
1,pretty but not worth the price tag overall it ...,0,0
2,great dress this dress look and feel classy an...,1,1
3,great feature love the pocket and back of dres...,1,1
4,white be not white this be a very nice jacket ...,1,1
5,love this jumpsuit I m in love with this jumps...,1,1
6,,1,1
7,versatile piece to finish out your look love h...,1,1
8,hard fit the color on these be amazing I be 5 ...,1,1
9,pretty color but fit like a sack it make I loo...,0,0
